## Formulation of the Problem

We aim to predict the solutions for the following system of ODEs (in nondimensional form):

\begin{align*}
\frac{d\tilde{u}}{d\tilde{t}} &= \tilde{v}, \\
\frac{d\tilde{v}}{d\tilde{t}} &= \kappa(v_0 \tilde{t} - \tilde{u}) - \alpha\Big( f_0 + a \ln \tilde{v} + b \ln \tilde{\theta} \Big), \\
\frac{d\tilde{\theta}}{d\tilde{t}} &= -\tilde{v}\tilde{\theta} \ln(\tilde{v}\tilde{\theta}).
\end{align*}

The loss function in the PINN is composed of three terms:

- **Residual loss ($\text{MSE}_R$)**: Enforces that the neural network output satisfies the ODEs.
- **Boundary loss ($\text{MSE}_B$)**: Ensures the initial conditions are met.
- **Measurement loss ($\text{MSE}_m$)**: Penalizes the difference between the network output and the measured values.

The overall loss is given by:


\begin{align*}
\text{MSE} &= \text{MSE}_R + \text{MSE}_B + \text{MSE}_m \\
\text{MSE}_R &= \frac{1}{N_R} \sum_{i=1}^{N_R} \Big( \left| \dot{u}(t_i, \varphi) - v(t_i, \varphi) \right|^2 
  + \left| \dot{v}(t_i, \varphi) - \kappa(v_0 t_i - u(t_i, \varphi)) + \alpha\Big(f_0 + a \ln v(t_i, \varphi) + b \ln \theta(t_i, \varphi)\Big) \right|^2 
  + \left| \dot{\theta}(t_i, \varphi) + v(t_i, \varphi) \theta(t_i, \varphi) \ln\left(v(t_i, \varphi)\theta(t_i, \varphi)\right) \right|^2 \Big) \\
  \text{MSE}_B &= \frac{1}{N_B} \sum_{i=1}^{N_B} \Big( \left| u(0, \varphi) - u_0 \right|^2 + \left| v(0, \varphi) - v_0 \right|^2 
  + \left| \theta(0, \varphi) - \theta_0 \right|^2 \Big) \\\text{MSE}_m &= \frac{1}{N_m} \sum_{i=1}^{N_m} \Big( \left| u(t_i, \varphi) - u^*(t_i) \right|^2 + \left| v(t_i, \varphi) - v^*(t_i) \right|^2 
  + \left| \theta(t_i, \varphi) - \theta^*(t_i) \right|^2 \Big).
\end{align*}


Here, $\varphi$ denotes the trainable parameters of the neural network.

## Import Libraries

We begin by importing the necessary libraries including DeepXDE, NumPy, Matplotlib, Pandas, and TensorFlow (via DeepXDE's backend).

In [1]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import integrate
from deepxde.backend import tf

Using backend: tensorflow.compat.v1



Instructions for updating:
non-resource variables are not supported in the long term



## Dataset Variables Explanation

The dataset (CSV file) contains the following columns:

- **`Var1`**: Time (independent variable, spanning from 0 to 100 seconds).
- **`y1_1`**: $\tilde{u}$ (slip) – the displacement of the block in the spring-block slider model.
- **`y1_2`**: $\tilde{v}$ (slip rate) – the time derivative of slip (velocity).
- **`y1_3`**: $\theta\$ (state variable) – a variable from the rate-and-state friction law.

These measurements will be used to enforce the measurement loss in the PINN framework.

In [ ]:
# Read the dataset (we take the first 10,000 data points)
raw = pd.read_csv('./../Dataset/sbm1.csv')
raw = raw[0:10000]

# Extract columns from the dataset
observe_t = raw['Var1']
u_ext = raw['y1_1']
v_ext = raw['y1_2']
theta_ext = raw['y1_3']

## Interpolating Data and Plotting Measurements

We interpolate the full dataset to obtain **25 equidistant measurement points** (from time 0 to 100 seconds). 
These sparse measurements (\(u^*, v^*, \theta^*\)) will be used as observed data in our PINN training.

In [ ]:
t_int = np.linspace(0, 100, 25)

u_int = np.interp(t_int, observe_t.values.reshape(-1), u_ext.values.reshape(-1))
v_int = np.interp(t_int, observe_t.values.reshape(-1), v_ext.values.reshape(-1))
theta_int = np.interp(t_int, observe_t.values.reshape(-1), theta_ext.values.reshape(-1))

plt.figure(figsize=(8, 6))
plt.plot(observe_t, u_ext, label="Full Slip (u)")
plt.plot(observe_t, v_ext, label="Full Slip Rate (v)")
plt.plot(observe_t, theta_ext, label= r"Full State (\(\theta\))")

plt.scatter(t_int, u_int, label="Measured Slip", color='red')
plt.scatter(t_int, v_int, label="Measured Slip Rate", color='orange')
plt.scatter(t_int, theta_int, label= r"Measured State (\(\theta\))", color='green')

plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()
plt.grid(True)
plt.show()

# Reshape the interpolated arrays for DeepXDE (each as a column vector)
observe_t = t_int.reshape((-1, 1))
u_ext = u_int.reshape((-1, 1))
v_ext = v_int.reshape((-1, 1))
theta_ext = theta_int.reshape((-1, 1))

## Defining Measurement Boundary Conditions

We define the measurement boundary conditions using DeepXDE's `PointSetBC` class. 
These boundary conditions force the neural network solution to match the measured values 
at the specified time points for each of the three components: \(u\), \(v\), and \(\theta\).

In [ ]:
observe_y0 = dde.icbc.PointSetBC(observe_t, u_ext, component=0)
observe_y1 = dde.icbc.PointSetBC(observe_t, v_ext, component=1)
observe_y2 = dde.icbc.PointSetBC(observe_t, theta_ext, component=2)

In [ ]:
# Define physical parameters
alpha = 9.81
kappa = 0.25
v0 = 1
f0 = 0.2
a = 0.2
b = 0.3

In [ ]:
def ode_system(x, y):
    # y is a tensor with three columns corresponding to u, v, and theta
    u = y[:, 0:1]
    v = y[:, 1:2]
    theta = y[:, 2:3]

    # Compute time derivatives using automatic differentiation
    du_t = dde.grad.jacobian(y, x, i=0)
    dv_t = dde.grad.jacobian(y, x, i=1)
    dtheta_t = dde.grad.jacobian(y, x, i=2)

    # Use tf.clip_by_value to prevent taking the logarithm of non-positive values
    v_clip = tf.clip_by_value(v, 1e-6, 13)
    theta_clip = tf.clip_by_value(theta, 1e-6, 11)

    # Define the residuals for each ODE
    res_u = du_t - v_clip
    res_v = dv_t - (kappa * (v0 * x - u) - alpha * (f0 + a * tf.math.log(v_clip) + b * tf.math.log(theta_clip)))
    res_theta = dtheta_t + v_clip * theta_clip * tf.math.log(v_clip * theta_clip)

    return [res_u, res_v, res_theta]

## Compile and Train the PINN Model

We now set up the PINN to solve the ODE system. 

- **Geometry**: We define the time domain as \([0, 100]\).
- **Data**: We use the defined ODE system along with the measurement boundary conditions. 
  The model uses 20,000 residual points for enforcing the physics and 25 measurement points for data loss.
- **Neural Network Architecture**: The feed-forward network has 6 hidden layers of 64 neurons each, 
  with hyperbolic tangent (tanh) as the activation function. The network has 1 input (time) and 3 outputs 
  (\(\tilde{u}\), \(\tilde{v}\), and \(\tilde{\theta}\)).
- **Output Transform**: We apply a transform to help the network better capture the behavior near the boundaries.
- **Training**: The model is compiled with the Adam optimizer (learning rate = 0.0001) and trained for 50,000 iterations.

In [ ]:
geom = dde.geometry.TimeDomain(0, 100)

In [ ]:
data = dde.data.PDE(
    geom,
    ode_system,
    [observe_y0, observe_y1, observe_y2],
    20000,
    0,
    num_test=3000
)

In [ ]:
layer_size = [1] + [64] * 6 + [3]
activation = "tanh"
initializer = "Glorot normal"
net = dde.nn.FNN(layer_size, activation, initializer)

In [ ]:
def output_transform(t, y):
    # Modify the raw network output to better match the expected behavior at the boundaries
    y1 = y[:, 0:1]
    y2 = y[:, 1:2]
    y3 = y[:, 2:3]
    
    # The transform shifts the predictions to approximately match the initial conditions:
    # u -> tanh(t)*y1 + 1, v -> tanh(t)*y2 + 0.5, theta -> tanh(t)*y3 + 1
    return tf.concat([y1 * tf.tanh(t) + 1,
                      y2 * tf.tanh(t) + 0.5,
                      y3 * tf.tanh(t) + 1], axis=1)

net.apply_output_transform(output_transform)

In [ ]:
model = dde.Model(data, net)
model.compile(
    "adam",
    lr=0.0001,
    loss_weights=[1, 1, 1, 1, 1, 1]
)

# Create output directory for saving checkpoints
path = "./../output/Model/model"
os.makedirs(path, exist_ok=True)
checkpoint_path = os.path.join(path, "model.ckpt")
checker = dde.callbacks.ModelCheckpoint(
      checkpoint_path, save_better_only=True, period=50
  )

# Train the model for 50,000 iterations (using 'epochs' which is equivalent to iterations here)
losshistory, train_state = model.train(epochs=50000)

## Prediction and Plotting

After training the PINN, we predict the solution over the time domain and plot the network predictions 
against the true values (from the dataset). This visualization helps assess how well the PINN has learned 
the underlying dynamics of the nonlinear frictional system.

In [ ]:
observe_t = raw['Var1']
u_ext = raw['y1_1']
v_ext = raw['y1_2']
theta_ext = raw['y1_3']

plt.figure(figsize=(8, 6))
plt.xlabel("Time")
plt.ylabel("y")

plt.plot(observe_t, u_ext, color="black", label="True u")
plt.plot(observe_t, v_ext, color="blue", label="True v")
plt.plot(observe_t, theta_ext, color="brown", label=r'True $\theta$')

t = np.linspace(0, 100, 10000).reshape((-1, 1))
sol_pred = model.predict(t)
u_pred = sol_pred[:, 0:1]
v_pred = sol_pred[:, 1:2]
theta_pred = sol_pred[:, 2:3]

plt.plot(t, u_pred, color="red", linestyle="dashed", label="Predict u")
plt.plot(t, v_pred, color="orange", linestyle="dashed", label="Predict v")
plt.plot(t, theta_pred, color="green", linestyle="dashed", label=r"Predict $\theta$")
plt.legend()
plt.grid(True)
plt.savefig('./../output/pred.png')
plt.show()